In [2]:
import numpy as np
import math
from collections import Counter
from time import sleep

NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
MINMAX_DEPTH = 2
MONTECARLO_SAMPLES = 5 #number of times "play out" is performed at each call of the simulation step 
MONTECARLO_STEPS = 20 #number of times MCTS sequence (select+expand+simulate+backprop) is iterated. However MCTS is called once for each possible move in montecarlo only mode 
EVAL_MODE = 1 #0 for minmax + montecarlo, 1 for montecarlo only

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"


Basic Functions

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

MinMax

In [4]:
def winrate_to_score(winrate, player):
    if player == 1:
        return winrate*2 -1
    else:
        return (1-winrate)*2 -1


def minmax(board, player, depth, a,b):
    if(four_in_a_row(board,-player)):
        #d=""
        #for i in range(depth):
        #    d= d+" > "
        #print(d+ f"terminal score:  {-player} moving player: {player}")
        return (-1,-player, depth)
    
    if(depth>= MINMAX_DEPTH):
        #score, _, __, ___ = mc_simulate(board,player)
        #d=""
        #for i in range(depth):
        #    d= d+" > "
        
        winrate = montecarlo_wrap(board, -player)
        score = winrate_to_score(winrate,-player)
        #print(d+ f"montecarlo score:  {score} winrate: {winrate} moving player: {player}")
        #print(board)
        
        return (-1,score, depth)

    evaluations = list()
    #print(valid_moves(board))
    cur_a = -math.inf
    cur_b = math.inf
    for m in valid_moves(board):
        play(board, m, player)
        val = minmax(board,-player, depth+1, cur_a, cur_b)
        evaluations.append((m,val[1], val[2]))
        take_back(board, m)
        if alpha_beta_stop(a,b,player, val[1]):
            #print(f"stopped a = {a} b = {b} cur val = {val[1]}, moving player: {player}")
            #print("cut!")
            break
        cur_a, cur_b=alpha_beta_update(cur_a,cur_b, player, val[1])
    
    #print(f"depth: {depth} player: {player} ab: {a} {b} cur_ab: {cur_a} {cur_b} evals: {evaluations} ")
    
    #biased to pick solution with least depth
    if player>0:
        ch=max(evaluations, key = lambda k: (k[1]*100, -k[2]) )
        #d=""
        #for i in range(depth):
        #    d= d+" > "
        #print(d+ f"moves: {evaluations} chosen move: {ch[0]} score:  {ch} moving player: {player}")
        return ch
    else:
        ch=min(evaluations, key=lambda k: (k[1]*100, k[2]))
        #d=""
        #for i in range(depth):
        #    d= d+" > "
        #print(d+ f"moves: {evaluations} chosen move: {ch[0]} score:  {ch} moving player: {player}")
        return ch

def alpha_beta_update(a,b, player, cur):
    if player <0 and cur < b:
        #print("update")
        return a, cur
    elif player > 0 and cur> a:
        #print("update"); 
        return cur, b 
    return a,b

def alpha_beta_stop(a, b, player, cur):
    if player <0 and cur <= a:
        return True
    elif player > 0 and cur >= b:
        return True 
    return False





Montecarlo

In [5]:
class node:
    def __init__(self, board, num, den, player, parent, move):
        self.board = np.copy(board)
        self.num = num #unutilized

        self.move = move #needed just for debugging
        self.winmin = 0
        self.winmax = 0
 
        self.den = den
        self.player=player
        self.parent=parent
        self.children=list()
        self.terminal = four_in_a_row(board, -player)
        self.n_children = 0
        self.terminated_childrend = 0
    
    def simulate(self):

        if self.terminal:
            #you represent a stronger simulation, give stronger feedback now because you're not going to be selected again
            draws=0
            if self.player==-1:
                winmax = MONTECARLO_SAMPLES*NUM_COLUMNS
                winmin = 0
                draws = 0
            elif self.player == 1:
                winmax = 0
                winmin = MONTECARLO_SAMPLES*NUM_COLUMNS
                draws = 0

        else:
            _,draws,winmax,winmin = mc_simulate(self.board, self.player)
        
        #update current node statistics
        self.winmin += winmin + 0.5* draws
        self.winmax += winmax + 0.5* draws
        
        self.den += winmax+winmin+draws

        #print(f"winmax {winmax} winmin {winmin} draws {draws}")
        #backpropagate evaluation
        if self.parent:
            #self.parent.backprop(winmax,winmin, draws, MONTECARLO_SAMPLES)
            self.parent.backprop(winmax+ 0.5* draws,winmin+ 0.5* draws, winmax+winmin+draws)
            
        

    def backprop(self,winmax,winmin,den):
        #print("bp")
        self.winmax+=winmax
        self.winmin +=winmin
    
        self.den += den
        if self.parent:
            self.parent.backprop(winmax,winmin, den)
    
    def expand(self):
        #print("expanding")
        #print(self.board)
        
        for m in valid_moves(self.board):
            
            play(self.board,m,self.player)
            c = node(self.board,0,0,-self.player, self,m)
            c.simulate()
            self.children.append(c)
            
            take_back(self.board, m)
            if(c.terminal):
                #the current state gives the opponent a win in 1 move, let parent node know they should not reach it and that algorithm should not waste time here
                #print(f"TERMINAL: previous move: {self.move} by {-self.player} winrate min: {self.winrate(-1)} winrate max: {self.winrate(1)}")
                #print(self.board)
                
                if self.parent:
                    #clear all your contribution so far
                    self.parent.backprop(-self.winmax,-self.winmin,-self.den)
                    #turn it into a complete win/defeat contribution
                    if self.player == -1:
                        self.parent.backprop(0, self.den, self.den)
                    else:
                        self.parent.backprop( self.den,0, self.den)

                if self.player == -1:
                    self.winmax = 0
                    self.winmin = self.den
                else:
                    self.winmin = 0
                    self.winmax = self.den
                self.terminal = True
                #print(f"TERMINAL AFTER: previous move: {self.move} by {-self.player} winrate min: {self.winrate(-1)} winrate max: {self.winrate(1)}")
               
                break
                
        #print(f"winrate for this {self.winrate()}")
        return self.children.copy()
    def winrate(self, player):
        if player == 1:
            return self.winmax/self.den
        elif player == -1:
            return self.winmin/self.den
        return 0.0
        #simplified UCT
        #return self.num/self.den
         

def mc_select(node):
    if node.terminal:
        return node
    if not node.children:
        return node
    if not list(filter(lambda c: c.terminal == False, node.children)):
        return node
    cur_best = max(filter(lambda c: c.terminal == False, node.children), key= lambda n: n.winrate(node.player))

    return mc_select(cur_best)

def mc_playout(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0



def mc_simulate(board, player):
    
    cnt = Counter(mc_playout(np.copy(board), player) for _ in range(MONTECARLO_SAMPLES))
    return (cnt[1] - cnt[-1]) / MONTECARLO_SAMPLES, cnt[0], cnt[1], cnt[-1]

def montecarlo_wrap(board, player, prev_move = -1):
    #define root node
    root = node(board, 0,0, -player, None,prev_move) #-player because it's their turn to move 
    if(root.terminal):
        return 1.0
    root.expand()

    #main loop
    for i in range(MONTECARLO_STEPS):
        #selection: get leaf by going through path with best winrates 
        next_node = mc_select(root)

        if next_node.terminal:
            break
        

        #expand the node (expands + simulation + backpropagation)
        children = next_node.expand()

    #if prev_move == 1:
        #tree_print(root, 0)
    #return winrate of root node
    return root.winrate(player) 

def tree_print(node, depth):
    d=""
    for i in range(depth):
        d= d+" > "
    print(d+f"previous move: {node.move} by {-node.player} winrate min: {node.winrate(-1)} winrate max: {node.winrate(1)} terminal: {node.terminal}")
    for child in node.children:
        tree_print(child, depth+1)

Evaluation wrapper

In [6]:
def montecarlo_only(board, player):
    #montecarlo evaluation for each possible move, then choose best move
    evaluations = list()
    print("ai is thinking aloud...")
    for m in valid_moves(board):
        play(board, m, player)
        winrate = montecarlo_wrap(board, player, m)
        evaluations.append((m,winrate))
        take_back(board, m)
        print(f"{m} gives {winrate} winrate")
    
    return max(evaluations, key = lambda k: k[1])
        

#returns: move, score. Score is in range (-1,1) if using minmax+montecarlo (EVAL_MODE = 0), a winrate in range (0,1) with montecarlo only 
def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return -1,1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1,1
    else:
        # Not terminal, let's simulate...
        if EVAL_MODE ==0:
            
            eval= minmax(board, player,0, -math.inf, math.inf)
            return eval[0], eval[1]
        elif EVAL_MODE == 1:
            eval = montecarlo_only(board, player)
            return eval[0], eval[1]
        else:
            print("wrong evaluation mode selected, check the EVAL_MODE constant")
            return -1,0


Example (used for debugging)

In [7]:


board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
#play(board, 5, 1)
#play(board, 2, -1)
#play(board, 0, -1)
#play(board,2,-1)
#board*=-1

print(board)
#eval_board(board, 1) uncommenting gives the output below. Both 2 and 5 force a loss to player -1




[[-1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


Simple Human vs AI handler (run to play)

In [8]:
def match(board):
    start= input("who will start? ('h' for human, 'a' for AI, default is human): ")
    ai=-1
    human = 1
    if(start == 'a'):
        
        human = -1
        ai = 1
    curplayer = 1

    while valid_moves(board):
        print(board)
        sleep(0.5)
        
        
        print(f"legal moves are {valid_moves(board)}")
        if curplayer == human:
            
            move = -1
            
            while not False:
                move = int(input("type your move:"))
                if move not in valid_moves(board):
                    print("illegal move!")
                else:
                    print(f"you chose {move}")
                    break
            play(board, move, curplayer)
            
        elif curplayer == ai:
            move = eval_board(board, curplayer)
            print(f"ai played {move[0]}")
            play(board,move[0], curplayer)
            
        
        if four_in_a_row(board, human):
            print(board)
            print("you won!")
            break
        elif four_in_a_row(board, ai):
            print(board)
            print("ai won!!!")
            break
        
        curplayer*=-1
        

board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
match(board)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
you chose 3
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
ai is thinking aloud...
0 gives 0.24765100671140938 winrate
1 gives 0.319047619047619 winrate
2 gives 0.391156462585034 winrate
3 gives 0.39933774834437086 winrate
4 gives 0.33537414965986395 winrate
5 gives 0.3578231292517007 winrate
6 gives 0.2593548387096774 winrate
ai played 3
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
you chose 4
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
ai is thinking aloud...
0 gives 0.